# Machine Learning HW1

## PM2.5 Prediction

In [2]:
import numpy as np
import pandas as pd

In [44]:
data = pd.read_csv('ml2020spring-hw1/train.csv', encoding = 'big5')
data.head(18)

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
5,2014/1/1,豐原,NO2,16,9.2,8.2,6.9,6.8,3.8,6.9,...,11,11,22,28,19,12,8.1,7,6.9,6
6,2014/1/1,豐原,NOx,17,9.8,8.7,8.6,8.5,5.3,8.8,...,14,13,25,30,21,13,9.7,8.6,8.7,7.5
7,2014/1/1,豐原,O3,16,30,27,23,24,28,24,...,65,64,51,34,33,34,37,38,38,36
8,2014/1/1,豐原,PM10,56,50,48,35,25,12,4,...,52,51,66,85,85,63,46,36,42,42
9,2014/1/1,豐原,PM2.5,26,39,36,35,31,28,25,...,36,45,42,49,45,44,41,30,24,13


In [45]:
data1 = data.iloc[:,3:]
data1[data1 == 'NR'] = 0
data1.shape

(4320, 24)

In [49]:
raw_data = data1.to_numpy()

### Extract Feature 1

将 $ 4320 * 18$ 的数据转为 $12 * 20 * 24 * 18 $ 的数据，12个月，每月20天，每天24小时，每小时18个特征。

必须分开12个月计算，每个月的时间不连续。

In [55]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:,day * 24 : (day+1) * 24] = raw_data[18 * (month * 20 + day) : 18 * (month * 20 + day + 1),:]
    month_data[month] = sample

In [72]:
# 每个月数据均为 18 * 480
month_data[11]

array([[ 23.  ,  23.  ,  23.  , ...,  13.  ,  13.  ,  13.  ],
       [  1.6 ,   1.7 ,   1.7 , ...,   1.8 ,   1.8 ,   1.8 ],
       [  0.22,   0.2 ,   0.18, ...,   0.51,   0.57,   0.56],
       ...,
       [ 93.  ,  50.  ,  99.  , ..., 118.  , 100.  , 105.  ],
       [  1.8 ,   2.1 ,   3.2 , ...,   1.5 ,   2.  ,   2.  ],
       [  1.3 ,   0.9 ,   1.  , ...,   1.6 ,   1.8 ,   2.  ]])

### Extract Feature 2
每个月连续时间480小时，每9个小时产生一组data，每月共471组连续数据。

In [83]:
x_data = np.empty([471 * 12, 18 * 9], dtype = float)
y_data = np.empty([471 * 12, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x_data[month * 471 + day * 24 + hour,:] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1)
            y_data[month * 471 + day * 24 + hour,:] = month_data[month][9,day * 24 + hour + 9]
print(x_data.shape)
print(y_data.shape)

(5652, 162)
(5652, 1)


### Normalization

In [78]:
x_mean = np.mean(x_data, axis = 0)
x_std = np.std(x_data, axis = 0)
x_mean.shape,x_std.shape

((162,), (162,))

In [82]:
for i in range(len(x_data)):
    for j in range(len(x_data[i])):
        if x_std[j]:
            x_data[i][j] = (x_data[i][j] - x_mean[j]) / x_std[j]
x_data

(array([[-3.80191926, -3.80172898, -3.80098934, ..., -1.35805482,
         -1.35915993, -2.6791547 ],
        [-3.80191926, -3.80172898, -3.82626312, ..., -1.35805482,
         -2.67575933, -2.85495044],
        [-3.80191926, -3.82702726, -3.8515369 , ..., -2.67546148,
         -2.85130591, -2.41546108],
        ...,
        [-3.72597039, -3.70053589, -3.67462047, ..., -2.14849882,
         -1.88579969, -1.97597173],
        [-3.70065411, -3.67523761, -3.69989424, ..., -1.88501748,
         -1.97357298, -1.71227811],
        [-3.67533782, -3.70053589, -3.72516802, ..., -1.97284459,
         -1.7102531 , -1.53648237]]), (5652, 162))

In [ ]:
### Split Validation Set